Import Packages (Same as before)

In [1]:
###################### STEP 1 ###################### 
#import packages
import requests
import time
import os
import pandas as pd
import numpy as np
import random
import re
import regex

# !pip3 install -U spacy
import spacy

from time import sleep
from os import path
from pandas import DataFrame
from bs4 import BeautifulSoup
from random import randint

# import selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.common.exceptions import NoSuchElementException
# from selenium.common.exceptions import ElementClickInterceptedException
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains

# set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# specify file path of chromedriver
# s = Service('/home/himi/Documents/RCC/rent_scraper/chromedriver')

snapshot_links, name, owner, address, price_range, bed_range, amenities, contact, property_link = [], [], [], [], [], [], [], [], []

Get snapshot links (Same as before)

In [8]:
###################### STEP 2 ###################### 

# open chrome
browser = webdriver.Chrome(service=s)

# define action
action = ActionChains(browser)

# list of years to loop over
years = ["2015"]


# scrape snapshot links
for y in years:
    
    # open link
    print("".join(["start scraping blue snapshots from ", y]))
    yearpage = "https://web.archive.org/web/" + y + "0000000000*/https://www.apartments.com/chicago-il/"
    browser.get(yearpage)
    time.sleep(10)
    
    # find all blue snapshots
    blues = browser.find_elements("xpath","//*[@class='measure s2xx']")
    time.sleep(5)
    
    # hover over blue snapshot and fetch snapshot link elements
    for blue in blues:
        day = blue.find_element("xpath","../..")
        time.sleep(2)
        action.move_to_element(day).perform()
        time.sleep(2)
        popup = browser.find_element("xpath","//*[@class='popup-of-day']")
        time.sleep(2)
        action.move_to_element(popup).perform()
        time.sleep(2)
        hrefs = browser.find_elements("xpath","//*[@class='s2xx snapshot-link']")
        
        # save each link and append to list
        for href in hrefs: 
            link = href.get_attribute("href")
            time.sleep(2)
            snapshot_links.append(link)

    # done with year
    print("".join(["* done with ", y]))
    
# close chrome            
browser.quit()

start scraping blue snapshots from 2015
* done with 2015


Function to get all the contents from a particular webpage

In [2]:
def get_contents(soup, content_text):
  try:
    parents_blacklist=['[document]','html','head',
                       'style','script','body',
                       'section','tr',
                       'td','label','ul','header',
                       'aside']
    content=''
    text=soup.find_all(text=True)

    
    for t in text:
        if t.parent.name not in parents_blacklist and len(t) > 5:
            content=content+t+' '
    content_text.append(content)
  except Exception:
    content_text.append('')
    pass



Function to get phone numbers, address and prices using regular expressions

In [3]:
def get_regex_data(webpage):
    page=requests.get(webpage)
    soup=BeautifulSoup(page.text,'html.parser')
    content_text = []
    get_contents(soup, content_text)
    contacts = re.findall(r'\(?[0-9]{3}\)?\/?\.?-?\s?[0-9]{3}-?\.?-?\s?[0-9]{4}', content_text[0])
    address = regex.findall(r'[\s\n]*[1-9][0-9]{,4} (?:[A-Z][a-zA-Z]*,?\s*){,5} Chicago,?[\w\s,]{,3}\s*[0-9]{,6}?', content_text[0])
    address = [addi.strip() for addi in address]
    prices = []
    prices = re.findall(r'\$[1-9]?[0-9]{,3},?[0-9]{,3}\s*-\s*\$?[1-9]?[0-9]{,3},?[0-9]{,3}', content_text[0])
    print("\nContacts are:", contacts)
    print("\nAddresses are:", address)
    print("\nPrices are:" , prices)

In [4]:
page_2015 ="https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/"
page_2022 = "https://web.archive.org/web/20220819053652/https://www.apartments.com/chicago-il/"

print("For 2015:")
get_regex_data(page_2015)

print("For 2022:")
get_regex_data(page_2022)

For 2015:

Contacts are: ['312-288-8069', '312-219-9513', '773-904-9571', '866-611-2494', '866-599-7868', '312-985-0261', '866-933-3491', '866-610-8317', '773-273-6395', '312-462-0403', '773-675-1163', '312-462-1903', '773-492-6656', '855-254-1372', '844-274-9788', '773-649-5501', '872-206-3038', '844-406-5855', '844-443-8919', '708-546-4173', '773-432-4830', '312-462-1938', '312-646-4941', '312-646-4953', '773-358-4744']

Addresses are: ['360 E South Water St Chicago', '71 W Hubbard St Chicago', '345 E Wacker Dr Chicago', '5050 S Lake Shore Dr Chicago', '400 E South Water St Chicago', '225 N Columbus Dr Chicago', '1130 S Michigan Ave Chicago', '200 E Illinois St Chicago', '1350 N Lake Shore Dr Chicago', '750 N Rush St Chicago', '850 N Lake Shore Dr Chicago', '220 W Illinois St Chicago', '4740 S Woodlawn Ave Chicago', '445 E Illinois St Chicago', '5009 N Sheridan Rd Chicago', '805 N LaSalle St Chicago', '601 W Jackson Blvd Chicago', '111 W Wacker Dr Chicago', '5140 S Hyde Park Blvd Chi

In [10]:
links = ['https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220612023135/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220321160908/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220121035604/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220426234339/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220403132436/https://www.apartments.com/chicago-il/']

In [12]:
for link in links:
    get_regex_data(link)


Contacts are: ['312-288-8069', '312-219-9513', '773-904-9571', '866-611-2494', '866-599-7868', '312-985-0261', '866-933-3491', '866-610-8317', '773-273-6395', '312-462-0403', '773-675-1163', '312-462-1903', '773-492-6656', '855-254-1372', '844-274-9788', '773-649-5501', '872-206-3038', '844-406-5855', '844-443-8919', '708-546-4173', '773-432-4830', '312-462-1938', '312-646-4941', '312-646-4953', '773-358-4744']

Addresses are: ['360 E South Water St Chicago', '71 W Hubbard St Chicago', '345 E Wacker Dr Chicago', '5050 S Lake Shore Dr Chicago', '400 E South Water St Chicago', '225 N Columbus Dr Chicago', '1130 S Michigan Ave Chicago', '200 E Illinois St Chicago', '1350 N Lake Shore Dr Chicago', '750 N Rush St Chicago', '850 N Lake Shore Dr Chicago', '220 W Illinois St Chicago', '4740 S Woodlawn Ave Chicago', '445 E Illinois St Chicago', '5009 N Sheridan Rd Chicago', '805 N LaSalle St Chicago', '601 W Jackson Blvd Chicago', '111 W Wacker Dr Chicago', '5140 S Hyde Park Blvd Chicago', '85


Contacts are: ['(773) 535-0040', '(773) 534-5840', '(773) 534-9190', '(773) 534-7790', '(773) 534-5720', '(773) 535-0040', '(773) 534-5840', '(773) 534-9190', '(773) 534-7790', '(773) 534-5720', '(773) 534-8600', '(773) 534-0034', '(773) 534-3954', '(773) 697-2216', '(773) 534-6400', '(847) 440-3110', '(844) 295-8794', '(224) 203-2815', '(866) 794-5357', '(773) 917-0650', '(872) 285-5421', '(224) 215-0020', '(224) 326-0829', '(844) 294-8524', '(312) 815-5216', '(844) 264-5425', '(844) 288-1201', '(844) 694-6517', '(844) 410-6603', '(844) 402-4750', '(872) 873-3137', '(844) 417-3242', '(947) 252-6867', '(833) 218-5816', '(847) 979-4467', '(844) 402-4562', '(844) 489-9630', '(312) 462-0329', '(844) 859-7954', '(847) 447-2282']

Addresses are: ['930 W Altgeld St, Chicago, IL', '1457 N Halsted St, Chicago, IL', '5035 S East End Ave, Chicago, IL', '5441 NE NEast River Rd, Chicago, IL', '5036 N Sheridan Rd Chicago, IL', '369 W Grand Ave, Chicago, IL', '1212 N State Pky, Chicago, IL', '730 W